# Mediationsprototype 2.2 
 Version 2 Teil 2 Anonymisierung Presidio 

daten /ppm werden nicht im Bitbucket gehalten (Privacy)
anonymizierte Daten  in /anon  )


In [ ]:
# Module install 
#!pip install presidio_analyzer presidio_anonymizer
#!python -m spacy download de_core_news_lg

In [ ]:
# imports
import os
import re
from striprtf.striprtf import rtf_to_text

from presidio_analyzer import AnalyzerEngine, RecognizerRegistry, PatternRecognizer
from presidio_analyzer.nlp_engine import NlpEngineProvider, TransformersNlpEngine


#from transformers import pipeline
from presidio_anonymizer import AnonymizerEngine, DeanonymizeEngine, OperatorConfig
from presidio_anonymizer.operators import Operator, OperatorType

from typing import Dict
from pprint import pprint

import transformers
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer, AutoModelForTokenClassification


## Vorbereiten der rtf
Die Quellscripte werden von technischen Steuerzeichen befreite und als UTF8 im txt Format vorbereitet.
Es wird für jede Sitzung eine Datei erstellt. _S1.txt .. _S6.txt  

In [ ]:
# Definiere die Verzeichnisse und Dateinamen
Fall= "F8"
Sitzungsanzahl = 5
input_dir = fr'ppm\{Fall}'  # Achte darauf, dass dieser Pfad existiert und korrekt ist
output_dir = fr'ppm\{Fall}'
output_file_path = os.path.join(output_dir, '_S')

# Überprüfen, ob das Zielverzeichnis existiert, und es ggf. erstellen
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Liste der RTF-Dateinamen ausgeben
print(f"Suche nach RTF-Dateien in: {input_dir}")

# RTF-Dateien finden und deren Inhalt bereinigen
rtf_files = [f'S{i}.rtf' for i in range(1, Sitzungsanzahl+1)]  # Liste mit Dateinamen von S1.rtf bis S6.rtf
extracted_texts = []  # Sammlung für bereinigte Texte

i = 1
for rtf_file in rtf_files:
    file_path = os.path.join(input_dir, rtf_file)
    if os.path.isfile(file_path):
        print(f"Verarbeite Datei: {file_path}")
        with open(file_path, 'r', encoding='utf-8') as file:
            rtf_content = file.read()
            text = rtf_to_text(rtf_content)  # RTF-Steuerdaten entfernen
            cleaned_text = '\n'.join(filter(lambda x: x.strip(), text.splitlines()))
            #extracted_texts.append(cleaned_text)
            with open(f"{output_file_path}{i}.txt" , 'w', encoding='utf-8') as output_file:
                output_file.write(''.join(cleaned_text))
                print(f"Die Inhalte wurden erfolgreich in '{output_file_path}{i}' gespeichert.")
    else:
        print(f"Datei nicht gefunden: {file_path}")
    i += 1


# Bereinigte RTF-Inhalte in eine Ausgabedatei schreiben
if extracted_texts:
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write('\n'.join(extracted_texts))
    print(f"Die Inhalte wurden erfolgreich in '{output_file_path}'.txt gespeichert.")
else:
    print("Keine RTF-Inhalte zum Speichern gefunden.")

## PII Analyse und  Anonymisierung
Die Analyse erfolt mit den Frameworks Spacy, Transformer und MS Presidio mit verschiedenen Identifikationsmethoden. Zudem werden Manuell Identifizierte Eigennamen und Orte als Liste hinzugefügt, da Testreihen bestimmte Eigenamen unzuverlässig erkannt haben. Hier ist Finetuning und Training der Modelle möglich, allerdings nicht Fokus dieser Untersuchung. 

In [ ]:
# laden der Transkripte
text_list = []
for i in range(1, Sitzungsanzahl+1):  # Annahme: _S1.txt bis _S6.txt
    filename = f'_S{i}.txt'
    input_file_path = os.path.join(input_dir, filename)
    print(f"Transkript : {input_file_path} geladen")
    with open(input_file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        text_list.append(text)

## Konfiguration von Presidio 
hier werden zudem die bekannten Entitäten und die Ausnahmen definiert

In [ ]:
#download the Models
transformers_model = "domischwimmbeck/bert-base-german-cased-fine-tuned-ner"
#snapshot_download(repo_id=transformers_model)
# Instantiate to make sure it's downloaded during installation and not runtime
AutoTokenizer.from_pretrained(transformers_model)
AutoModelForTokenClassification.from_pretrained(transformers_model)


In [ ]:
# Presidio-Konfiguration

# Bekannte Namen und Ort der Mediationen hier 
known_names_list = []
known_locations_list = []

#Whitelist --> alles was nicht als PII erkannt werden soll 
allow_list = ["Afrika","Tschüss","I","TASCHENGELD","TAG","ECHT","IHM","IHNEN","GEMEINSAM","GEGENWART","DADRIN","AUCH","BANK","Google","England","Europapark","Himalaya","Landschulheim","Schweiz","Tierpark","Instagram","GRUND","GANZ","ICH","FÜNF","MIR","Wischiwaschi","Weil","Uroma","Trulla","TAKTISCH","Ja","DU","Aha","SACHE","KINDER","Instagram","DOCH","Amazon","Corona","Flipchart","Flipcharts","Mensa Kids","YouTube","Stadt","B1","B2","I2","I1","Land","Scholli","Ähm","IMMER","SELBER","/","WÜRDEN","BOAH","VW","BIS","GUT","Schnieft","Boah"]           
PRESIDIO_ENTITIES = ["PERSON", "LOCATION", "ORGANIZATION", "EMAIL_ADDRESS", "IBAN_CODE", "PHONE_NUMBER"]

#spacy or transformers in configfile
NLP_CONFIG_FILE = "nlpconf.yml" 

# Create a PatternRecognizer for the deny list  
deny_list_recognizer_loc = PatternRecognizer(supported_entity="LOCATION", supported_language= "de", deny_list=known_locations_list)
deny_list_recognizer_per = PatternRecognizer(supported_entity="PERSON", supported_language= "de", deny_list=known_names_list)

registry = RecognizerRegistry()
registry.load_predefined_recognizers(languages=["de"])
registry.add_recognizer(deny_list_recognizer_loc)
registry.add_recognizer(deny_list_recognizer_per)

provider = NlpEngineProvider(conf_file=NLP_CONFIG_FILE)
nlp_engine = provider.create_engine()
analyzer = AnalyzerEngine(registry=registry, supported_languages=["de"], nlp_engine=nlp_engine)

In [ ]:
class InstanceCounterAnonymizer(Operator):
    """
    Anonymizer which replaces the entity value
    with an instance counter per entity.
    """

    REPLACING_FORMAT = "<{entity_type}_{index}>"

    def operate(self, text: str, params: Dict = None) -> str:
        """Anonymize the input text."""

        entity_type: str = params["entity_type"]

        # entity_mapping is a dict of dicts containing mappings per entity type
        entity_mapping: Dict[Dict:str] = params["entity_mapping"]

        entity_mapping_for_type = entity_mapping.get(entity_type)
        if not entity_mapping_for_type:
            new_text = self.REPLACING_FORMAT.format(
                entity_type=entity_type, index=0
            )
            entity_mapping[entity_type] = {}

        else:
            if text in entity_mapping_for_type:
                return entity_mapping_for_type[text]

            previous_index = self._get_last_index(entity_mapping_for_type)
            new_text = self.REPLACING_FORMAT.format(
                entity_type=entity_type, index=previous_index + 1
            )

        entity_mapping[entity_type][text] = new_text
        return new_text

    @staticmethod
    def _get_last_index(entity_mapping_for_type: Dict) -> int:
        """Get the last index for a given entity type."""

        def get_index(value: str) -> int:
            return int(value.split("_")[-1][:-1])

        indices = [get_index(v) for v in entity_mapping_for_type.values()]
        return max(indices)

    def validate(self, params: Dict = None) -> None:
        """Validate operator parameters."""

        if "entity_mapping" not in params:
            raise ValueError("An input Dict called `entity_mapping` is required.")
        if "entity_type" not in params:
            raise ValueError("An entity_type param is required.")

    def operator_name(self) -> str:
        return "entity_counter"

    def operator_type(self) -> OperatorType:
        return OperatorType.Anonymize

### Analyse und Ananomize Funktionen

In [ ]:
def anonymize_text_with_presidio2(text, analyzer_result):
    anonymizer_engine = AnonymizerEngine()
    anonymizer_engine.add_anonymizer(InstanceCounterAnonymizer)
    global entity_mapping
    anonymized_result = anonymizer_engine.anonymize(text=text, analyzer_results= analyzer_result,
                                          operators={ "DEFAULT": OperatorConfig(
                                                "entity_counter", {"entity_mapping": entity_mapping})})
    anonymized_text = anonymized_result.text
    return anonymized_text

### Ausführen der Analysen und Anonymisierungen

In [ ]:
#overwrite textlist
#text_list = ["Bernd wohnt in Berlin. Max Mustermann arbeitet bei Bosch und wohnt in (Karlsruhe?). in der Auenstrasse 3. Max hat aber auch Verwandte in Berlin."]
# Create a mapping between entity types and counters
entity_mapping = dict()
anonymized_text_presidio_list = []
global CountFind
CountFind = 0
for text in text_list:
    #presidio
    analyser_pres = analyzer.analyze(text=text, language="de", entities=PRESIDIO_ENTITIES, score_threshold=0.5,return_decision_process = True, allow_list = allow_list)
    CountFind += len(analyser_pres)
    #for results in analyser_pres:
        #print(f"Finding {results}  mit {results.analysis_explanation}")
    anonymized_text_presidio_list.append(anonymize_text_with_presidio2(text, analyser_pres))
    #print(anonymized_text_presidio_list[:200])

### Auswertung und Speichern

In [ ]:
for index,anon_text in enumerate(anonymized_text_presidio_list):
    print(f"\nAnonymisierter Text (Presidio){index+1}:\n{anon_text[:200]}")

print(f"mapping {CountFind}:\n")

# Festelegen des Verzeihnisses für Produktivumgebung
output_dir = fr'anon\{Fall}'
pprint(entity_mapping, indent=2)
#save
for index,anon_text in enumerate(anonymized_text_presidio_list):
    anon_output_file_path = os.path.join(output_dir, f'{Fall}_{index+1}.txt')        
    with open(anon_output_file_path, 'w', encoding='utf-8') as file:    
        file.write(anon_text)
print("Anonymisierte Transkriptionen gespeichert") 